In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from my_functions import z_NB, smooth_hist

In [ ]:
# The selection
selection = pd.read_csv('csv/selection_first_kinda.csv')
selection_last = pd.read_csv('csv/selection.csv')

path_to_files = '/home/alberto/almacen/Selected_LAEs/First_visual_sel'

# ATT inspection
vi_att = pd.read_csv(f'{path_to_files}/inspeccion_AT.csv')
score_att = vi_att.to_numpy()[:, 1].astype(int)
# SGL inspection
vi_sgl = pd.read_csv(f'{path_to_files}/Sid_classification_noInfo_iter2.csv')
score_sgl = vi_sgl.to_numpy()[:, 1].astype(int)
# AFS inspection
vi_afs = pd.read_csv(f'{path_to_files}/class_afs.csv')
score_afs = vi_afs.to_numpy()[:, 1].astype(int)
# PAM inspection
vi_pam = pd.read_csv(f'{path_to_files}/inspection_pam.csv')
score_pam = vi_pam.to_numpy()[:, 0].astype(int)

files_list = list(vi_sgl.to_numpy()[:, 4])
src_list = [s.replace('J-NEP', 'caca') for s in files_list]
src_list = [s.replace('-', ' ') for s in src_list]
src_list = [s.replace('.', ' ') for s in src_list]
src_list = [s.split() for s in src_list]
src_list = np.array([int(s[2]) for s in src_list])

bad_NB_image = np.array([5, 19, 21, 34, 36, 38, 39, 52, 55, 139, 140, 171])
for sc in [score_att, score_sgl, score_afs, score_pam]:
    sc[bad_NB_image] = 0

selection_last.head()

In [ ]:
L_Arr = np.zeros_like(src_list).astype(float)
for i, src in enumerate(src_list):
    try:
        this_L = float(selection[selection['src'] == src]['L_lya'])
    except:
        this_L = 0.
    L_Arr[i] = this_L

mag = np.zeros_like(src_list).astype(float)
for i, src in enumerate(src_list):
    try:
        this_mag = float(selection[selection['src'] == src]['r'])
    except:
        this_mag = 0.
    mag[i] = this_mag

z_Arr = np.zeros_like(src_list).astype(float)
for i, src in enumerate(src_list):
    try:
        this_z = z_NB(float(selection[selection['src'] == src]['nb_sel']))
    except:
        this_z = 0.
    z_Arr[i] = this_z

nb_Arr = np.zeros_like(src_list).astype(float)
for i, src in enumerate(src_list):
    try:
        this_nb = float(selection[selection['src'] == src]['nb_sel'])
    except:
        this_nb = 0.
    nb_Arr[i] = this_nb

EW_Arr = np.zeros_like(src_list).astype(float)
for i, src in enumerate(src_list):
    try:
        this_ew = float(selection[selection['src'] == src]['EW_lya'])
    except:
        this_ew = 0.
    EW_Arr[i] = this_ew

puri2d = np.zeros_like(src_list).astype(float)
for i, src in enumerate(src_list):
    try:
        this_puri2d = float(selection_last[selection_last['src'] == src]['puri'])
    except:
        this_puri2d = 0.
    puri2d[i] = this_puri2d

xim = np.zeros_like(src_list).astype(float)
for i, src in enumerate(src_list):
    try:
        this_xim = float(selection[selection['src'] == src]['x_im'])
    except:
        this_xim = 0.
    xim[i] = this_xim

yim = np.zeros_like(src_list).astype(float)
for i, src in enumerate(src_list):
    try:
        this_yim = float(selection[selection['src'] == src]['y_im'])
    except:
        this_yim = 0.
    yim[i] = this_yim

tile_id = np.zeros_like(src_list).astype(float)
for i, src in enumerate(src_list):
    try:
        this_tid = float(selection[selection['src'] == src]['tile_id'])
    except:
        this_tid = 0.
    tile_id[i] = this_tid

In [ ]:
selection_last_src = selection_last['src'].to_numpy()
last_sel_mask = np.zeros_like(src_list).astype(bool)

for src in selection_last_src:
    if src in src_list:
        last_sel_mask[src_list == src] = True

In [ ]:
colors_list = ['black', 'tab:red', 'tab:red', 'tab:orange', 'tab:green', 'tab:blue']
nbs_list = [[1, 4], [4, 8], [8, 12], [12, 16], [16, 20], [20, 24], [1, 20]]

# score = np.mean([score_afs, score_att, score_sgl, score_pam], axis=0)
score = np.mean([score_afs, score_att], axis=0)
score_list = [score_att, score_afs, score_sgl, score_pam]
score_label_list = ['ATT', 'AFS', 'SGL', 'PAM']

# score = score_afs

for jjj, [nb_min, nb_max] in enumerate(nbs_list):
    fig, (ax, axx) = plt.subplots(2, figsize=(5, 4), sharex=True,
                                  gridspec_kw=dict(height_ratios=[1.5, 1]))

    extra_mask = (
        (nb_Arr >= nb_min) & (nb_Arr <= nb_max)
        & (L_Arr > 0)
        & last_sel_mask
        & (score > 0)
    )
    bins = np.linspace(42.5, 46, 8)
    bins_c = [bins[i : i + 2].sum() * 0.5 for i in range(len(bins) - 1)]

    for iii, c in enumerate(colors_list):
        this_mask = (np.round(score) == iii) & extra_mask
        ax.plot(L_Arr[this_mask], EW_Arr[this_mask], ls='', marker='o',
                markerfacecolor=c, markeredgecolor=c, markersize=4)
        axx.plot(L_Arr[this_mask], puri2d[this_mask], color=c, markersize=4,
                 marker='o', ls='')
    
    # good_hist = np.histogram(L_Arr[(score > 3) & extra_mask], bins)[0]
    # bad_hist = np.histogram(L_Arr[(score <= 3) & (score > 0) & extra_mask], bins)[0]
    for kkk, score_i in enumerate(score_list):
        good_hist, pc = smooth_hist(L_Arr[(score_i > 3) & extra_mask], 42.5, 46, 0.1, 0.5)
        bad_hist, pc = smooth_hist(L_Arr[(score_i <= 3) & extra_mask], 42.5, 46, 0.1, 0.5)

        puri_Arr = good_hist / (good_hist + bad_hist)
        puri_err_Arr = (good_hist / (good_hist + bad_hist) ** 2 +
                        good_hist ** 2 / (good_hist + bad_hist) ** 4 * good_hist) ** 0.5
        puri_err_Arr[~np.isfinite(puri_err_Arr)] = 0
        puri_Arr[~np.isfinite(puri_Arr)] = 0.
        axx.plot(pc, puri_Arr, linewidth=2, label=score_label_list[kkk])

    # Load puri from mocks
    if jjj != 6:
        puri_mock = np.load(f'npy/puri_1d_{nb_min}-{nb_max}.npy')
        bc_mock = np.load(f'npy/puri_1d_bc.npy')
        axx.plot(bc_mock, puri_mock, ls='-', marker='', color='dimgray')
    if jjj == 6:
        puri_mock = np.load(f'tmp/comb_puri_1.0.npy')
        axx.plot(bc_mock, puri_mock, ls='-', marker='', color='dimgray')
    
    axx.grid()
    axx.set_ylim(0, 1)
    # ax.set_ylim(2e1, 1e4)
    ax.set_xlim(42.5, 46)
    ax.set_ylim(10, 1e3)
    ax.set_ylabel(r'Ly$\alpha$EW$_0$ ($\AA$)')
    axx.set_ylabel('Purity')
    axx.set_xlabel(r'$\log L_{\mathrm{Ly}\alpha}$ (erg$^{-1}\,$s$^{-1}$)')
    axx.legend()

    for axi in [ax, axx]:
        axi.tick_params(which='both', direction='in')
        axi.yaxis.set_ticks_position('both')
        axi.xaxis.set_ticks_position('both')

    ax.set_yscale('log')

    fig.subplots_adjust(hspace=0.1)
    # fig.savefig(f'/home/alberto/Desktop/pres711/vi_puri_{jjj}.png', bbox_inches='tight',
    #             bbox_pad_inches=0)
    plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))


mask = (L_Arr > 0) & last_sel_mask & (score > 0)

puri2d_h, pc = smooth_hist(L_Arr[mask], 42.5, 46, 0.1, 0.5,
                           weights=puri2d[mask])
L_Arr_h, pc = smooth_hist(L_Arr[mask], 42.5, 46, 0.1, 0.5)

mean_puri2d = puri2d_h / L_Arr_h

for iii, c in enumerate(colors_list):
    this_mask = (np.round(score) == iii) & mask
    ax.plot(L_Arr[this_mask], puri2d[this_mask], ls='', marker='o',
            markerfacecolor=c, markeredgecolor=c, markersize=4)

for kkk, score_i in enumerate(score_list):
    good_hist, pc = smooth_hist(L_Arr[(score_i > 3) & mask], 42.5, 46, 0.1, 0.5)
    bad_hist, pc = smooth_hist(L_Arr[(score_i <= 3) & mask], 42.5, 46, 0.1, 0.5)

    puri_Arr = good_hist / (good_hist + bad_hist)
    puri_err_Arr = (good_hist / (good_hist + bad_hist) ** 2 +
                    good_hist ** 2 / (good_hist + bad_hist) ** 4 * good_hist) ** 0.5
    puri_err_Arr[~np.isfinite(puri_err_Arr)] = 0
    puri_Arr[~np.isfinite(puri_Arr)] = 0.
    ax.plot(pc, puri_Arr, linewidth=2, label=score_label_list[kkk], c=f'C{kkk}')

ax.plot(pc, mean_puri2d, linewidth=2, c='k', ls='--', label='Mean 2D purity')

ax.set_ylim(0, 1)
ax.set_xlim(43, 45.5)
ax.set_xlabel(r'$\log L_{\mathrm{Ly}\alpha}$ (erg$^{-1}\,$s$^{-1}$)')
ax.set_ylabel('Purity')
ax.legend()

plt.show()

In [ ]:
for iii, score_i in enumerate(score_list):
    fig, (ax, axx) = plt.subplots(2, figsize=(5, 4), sharex=True,
                                gridspec_kw=dict(height_ratios=[1.5, 1]))

    mask = nb_Arr < 20
    print(mask.sum())

    bins = np.linspace(0, 1, 15)
    bins_c = [bins[i : i + 2].sum() * 0.5 for i in range(len(bins) - 1)]

    good_h, _ = ax.hist(puri2d[(score_i > 3) & mask], bins,
                        histtype='step', color='green')[:2]
    bad_h, _ = ax.hist(puri2d[(score_i <= 3) & (score_i > 0) & (L_Arr > 0) & mask],
                        bins, histtype='step', color='red')[:2]

    axx.plot(bins_c, good_h / (good_h + bad_h), c='k')
    axx.plot([0, 1], [0, 1], ls='--', c='dimgray')
    axx.set_ylim(0, 1)
    axx.set_xlim(0, 1)
    ax.set_title(score_label_list[iii])

    for axi in [ax, axx]:
        axi.tick_params(which='both', direction='in')
        axi.yaxis.set_ticks_position('both')
        axi.xaxis.set_ticks_position('both')

    fig.subplots_adjust(hspace=0.07)
    plt.show()

In [ ]:
from load_jpas_catalogs import load_minijpas_jnep

pm_flx, pm_err, x_im, y_im, _, number, starprob, spCl,\
        photoz, photoz_chi_best, photoz_odds = load_minijpas_jnep(selection=True)

pz = photoz[src_list]
odds = photoz_odds[src_list]
chi_best = photoz_chi_best[src_list]

In [ ]:
tile_ids = [2241, 2243, 2406, 2470, 2520]
tile_names = [f'AEGIS00{i}' for i in range(1, 5)] + ['J-NEP']
colors_list = ['black', 'tab:red', 'tab:red', 'tab:red', 'tab:green', 'tab:blue']

# score = score_att

for tid, name in zip(tile_ids, tile_names):
    fig, ax = plt.subplots(figsize=(5, 5))

    print(sum(tile_id == tid))
    for iii, c in enumerate(colors_list):
        this_mask = (np.round(score) == iii) & (tile_id == tid) & (mag < 24)
        ax.plot(xim[this_mask], yim[this_mask], ls='', marker='o',
                markerfacecolor=c, markeredgecolor=c, markersize=5)
        
        texts = [f'{s:0.2f}' for s in pz[this_mask]]
        for xim_i, yim_i, pz_i in zip(xim[this_mask], yim[this_mask], texts):
            ax.text(xim_i, yim_i, pz_i)
        ax.set_title(name)

    plt.show()

In [ ]:
df = pd.read_csv('/home/alberto/almacen/Selected_LAEs/First_visual_sel/inspeccion_AT_2.0.csv')
score_att_2 = df.to_numpy()[:, 1][:-2].astype(int)

score_2_dict = {
    1: 'Lya QSO',
    2: 'Contaminant QSO',
    3: 'Galaxy',
    4: 'I don\'t know'
}

fig, ax = plt.subplots(figsize=(5, 4))

for sc, lb in score_2_dict.items():
    ax.hist(selection_last['L_lya'][score_att_2 == sc], label=lb,
            histtype='step', linewidth=2, linestyle=('--' if sc == 4 else '-'))
ax.legend()

plt.show()

fig, ax = plt.subplots(figsize=(5, 4))

for sc, lb in score_2_dict.items():
    ax.hist(starprob[selection_last['src']][score_att_2 == sc], label=lb,
            histtype='step', linewidth=2, linestyle=('--' if sc == 4 else '-'))
ax.legend(loc=9)

plt.show()

In [ ]:
sum((selection_last['r'] < 23))